import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=900
cls_num=2
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=WormsTwoClass0(i)

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (77, 901)
first_test_shape= (181, 901)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [33 44]
max(train_feature_Altitude)= 4.1961
min(train_feature_Altitude)= -4.8873
first_train_sample=
 [ 1.        -0.77859   -0.74444   -0.72546   -0.70269   -0.71408
 -0.72167   -0.73305   -0.71787   -0.73685   -0.76341   -0.77479
 -0.76341   -0.74823   -0.72546   -0.73305   -0.73305   -0.74064
 -0.74444   -0.76341   -0.7672    -0.76341   -0.75582   -0.74823
 -0.74823   -0.75582   -0.74823   -0.75962   -0.75203   -0.74823
 -0.74823   -0.73305   -0.73305   -0.74064   -0.771     -0.80895
 -0.83931   -0.86587   -0.87725   -0.87346   -0.919     -0.86207
 -0.86207   -0.85069   -0.82792   -0.85069   -0.78238   -0.7672
 -0.74444   -0.72546   -0.75203   -0.6951    -0.6951    -0.72167
 -0.72926   -0.74444   -0.74444   -0.79756   -0.75203   -0.67233
 -0.64577   -0.67613   -0.65716   -0.61541   -0.63818   -0.63439
 -0.59264   -0.61162   -0.5547    -0.42567   -0.30044   -0.21316
 -0.22455 

In [3]:
np.shape(ecg)

(66, 901)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 33	2 44	
 max = 44


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (33, 901)
Up to class  2 train shape =  (77, 901)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 33 44]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.1375
min magnitude class 1  =  -4.037800000000001
after normalizing >>
max magnitude class 1  =  0.9870973424048264
min magnitude class 1  =  -0.8129555012440277

 cls 2  >> 
max magnitude class 2  =  4.8591000000000015
min magnitude class 2  =  -4.3114
after normalizing >>
max magnitude class 2  =  1.1459805799590468
min magnitude class 2  =  -0.8731972609375345
each class and its windows =
1 76	2 105	
 max instance in classes of test windows = 105


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.8731972609375345
max tst = 1.1459805799590468


In [14]:
len(wndws_test)

181

In [15]:
np.shape(wndws_test)

(181, 901)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(33, 901)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (77, 900)
y_train =>  (77,)
X_test  =>  (181, 900)
y_test  =>  (181,)
X_valid  =>  (77, 900)
y_valid  =>  (77,)


y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [23]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 1.0459 - accuracy: 0.5455 - val_loss: 1.2042 - val_accuracy: 0.5714
Epoch 2/1000
1/1 [==============================] - 0s 45ms/step - loss: 29.9118 - accuracy: 0.5714 - val_loss: 1.0139 - val_accuracy: 0.4286
Epoch 3/1000
1/1 [==============================] - 0s 43ms/step - loss: 5.8295 - accuracy: 0.4286 - val_loss: 1.0770 - val_accuracy: 0.4286
Epoch 4/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.9569 - accuracy: 0.4286 - val_loss: 1.0449 - val_accuracy: 0.5714
Epoch 5/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.7100 - accuracy: 0.5714 - val_loss: 1.0587 - val_accuracy: 0.4286
Epoch 6/1000
1/1 [==============================] - 0s 45ms/step - loss: 1.0706 - accuracy: 0.4286 - val_loss: 1.0598 - val_accuracy: 0.5714
Epoch 7/1000
1/1 [==============================] - 0s 44ms/step - loss: 0.8933 - accuracy: 0.6104 - val_loss: 1.0034 - val_accuracy: 0.5714
Epoch 8/1000
1

Epoch 59/1000
1/1 [==============================] - 0s 45ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6846 - val_accuracy: 0.5714
Epoch 60/1000
1/1 [==============================] - 0s 44ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6791 - val_accuracy: 0.5714
Epoch 61/1000
1/1 [==============================] - 0s 44ms/step - loss: 9.9737e-04 - accuracy: 1.0000 - val_loss: 0.6746 - val_accuracy: 0.5714
Epoch 62/1000
1/1 [==============================] - 0s 41ms/step - loss: 8.1899e-04 - accuracy: 1.0000 - val_loss: 0.6713 - val_accuracy: 0.5714
Epoch 63/1000
1/1 [==============================] - 0s 46ms/step - loss: 5.7250e-04 - accuracy: 1.0000 - val_loss: 0.6691 - val_accuracy: 0.5714
Epoch 64/1000
1/1 [==============================] - 0s 46ms/step - loss: 4.6967e-04 - accuracy: 1.0000 - val_loss: 0.6673 - val_accuracy: 0.5714
Epoch 65/1000
1/1 [==============================] - 0s 45ms/step - loss: 3.7524e-04 - accuracy: 1.0000 - val_loss: 0.6661 - val_acc

1/1 [==============================] - 0s 51ms/step - loss: 7.6479e-06 - accuracy: 1.0000 - val_loss: 0.6472 - val_accuracy: 0.7532
Epoch 116/1000
1/1 [==============================] - 0s 50ms/step - loss: 6.9110e-06 - accuracy: 1.0000 - val_loss: 0.6467 - val_accuracy: 0.7532
Epoch 117/1000
1/1 [==============================] - 0s 48ms/step - loss: 6.6974e-06 - accuracy: 1.0000 - val_loss: 0.6476 - val_accuracy: 0.7662
Epoch 118/1000
1/1 [==============================] - 0s 47ms/step - loss: 7.5535e-06 - accuracy: 1.0000 - val_loss: 0.6541 - val_accuracy: 0.6234
Epoch 119/1000
1/1 [==============================] - 0s 52ms/step - loss: 7.6340e-06 - accuracy: 1.0000 - val_loss: 0.6441 - val_accuracy: 0.7403
Epoch 120/1000
1/1 [==============================] - 0s 49ms/step - loss: 6.2964e-06 - accuracy: 1.0000 - val_loss: 0.6487 - val_accuracy: 0.7143
Epoch 121/1000
1/1 [==============================] - 0s 45ms/step - loss: 5.6895e-06 - accuracy: 1.0000 - val_loss: 0.6490 - val_acc

Epoch 171/1000
1/1 [==============================] - 0s 46ms/step - loss: 3.8240e-07 - accuracy: 1.0000 - val_loss: 0.6725 - val_accuracy: 0.5065
Epoch 172/1000
1/1 [==============================] - 0s 45ms/step - loss: 4.3813e-07 - accuracy: 1.0000 - val_loss: 0.6902 - val_accuracy: 0.4805
Epoch 173/1000
1/1 [==============================] - 0s 46ms/step - loss: 3.2202e-07 - accuracy: 1.0000 - val_loss: 0.6738 - val_accuracy: 0.5065
Epoch 174/1000
1/1 [==============================] - 0s 46ms/step - loss: 3.4369e-07 - accuracy: 1.0000 - val_loss: 0.6884 - val_accuracy: 0.4805
Epoch 175/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.6429 - accuracy: 0.8052
np.shape(X_test)= (181, 900, 1)
Accuracy=  0.8051947951316833


In [24]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [25]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 0s 24ms/step
array([[18, 58],
       [21, 84]], dtype=int64)
              precision    recall  f1-score   support

           1       0.46      0.24      0.31        76
           2       0.59      0.80      0.68       105

    accuracy                           0.56       181
   macro avg       0.53      0.52      0.50       181
weighted avg       0.54      0.56      0.53       181



In [26]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 1.0305 - accuracy: 0.4416 - val_loss: 17.2957 - val_accuracy: 0.5714
Epoch 2/1000
1/1 [==============================] - 0s 70ms/step - loss: 473.6895 - accuracy: 0.5714 - val_loss: 0.9917 - val_accuracy: 0.4286
Epoch 3/1000
1/1 [==============================] - 0s 73ms/step - loss: 4.3910 - accuracy: 0.4286 - val_loss: 1.0723 - val_accuracy: 0.4286
Epoch 4/1000
1/1 [==============================] - 0s 75ms/step - loss: 1.5649 - accuracy: 0.4286 - val_loss: 1.0862 - val_accuracy: 0.5714
Epoch 5/1000
1/1 [==============================] - 0s 74ms/step - loss: 0.8529 - accuracy: 0.4286 - val_loss: 1.0414 - val_accuracy: 0.5714
Epoch 6/1000
1/1 [==============================] - 0s 60ms/step - loss: 0.9542 - accuracy: 0.5714 - val_loss: 1.0753 - val_accuracy: 0.5714
Epoch 7/1000
1/1 [==============================] - 0s 59ms/step - loss: 0.8302 - accuracy: 0.4286 - val_loss: 1.0549 - val_accuracy: 0.5714
Epoch 8/1000

Epoch 59/1000
1/1 [==============================] - 0s 62ms/step - loss: 0.5985 - accuracy: 0.8052 - val_loss: 0.8530 - val_accuracy: 0.5714
Epoch 60/1000
1/1 [==============================] - 0s 62ms/step - loss: 0.3361 - accuracy: 0.9740 - val_loss: 0.7434 - val_accuracy: 0.4286
Epoch 61/1000
1/1 [==============================] - 0s 61ms/step - loss: 0.9598 - accuracy: 0.5584 - val_loss: 0.7321 - val_accuracy: 0.5714
Epoch 62/1000
1/1 [==============================] - 0s 63ms/step - loss: 0.9234 - accuracy: 0.5974 - val_loss: 0.7878 - val_accuracy: 0.4286
Epoch 63/1000
1/1 [==============================] - 0s 63ms/step - loss: 0.5438 - accuracy: 0.6494 - val_loss: 0.7990 - val_accuracy: 0.5714
Epoch 64/1000
1/1 [==============================] - 0s 63ms/step - loss: 0.2653 - accuracy: 0.9870 - val_loss: 0.7741 - val_accuracy: 0.5714
Epoch 65/1000
1/1 [==============================] - 0s 62ms/step - loss: 0.1521 - accuracy: 0.9870 - val_loss: 0.7401 - val_accuracy: 0.5714
Epoch 

In [27]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 0s 15ms/step
array([[  1,  75],
       [  2, 103]], dtype=int64)
              precision    recall  f1-score   support

           1       0.33      0.01      0.03        76
           2       0.58      0.98      0.73       105

    accuracy                           0.57       181
   macro avg       0.46      0.50      0.38       181
weighted avg       0.48      0.57      0.43       181

